# Assignment

## Brief

Write the Python codes for the following questions.

## Instructions

Paste the answer as Python in the answer code section below each question.

### Question 1

Question: From the `movies` collection, return the documents with the `plot` that starts with `"war"` in acending order of released date, print only title, plot and released fields. Limit the result to 5.

Answer:

In [ ]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://<DB-USERNAME>:<DB-PASSWORD>@cluster0.c0yg5bx.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))


Pinged your deployment. You successfully connected to MongoDB!


In [5]:
db = client.sample_mflix
movies = db.movies
comments = db.comments

In [ ]:
for m in movies.find({"plot": {"$regex": "^war", "$options": "i"}},  # case-insensitive
                     {"title": 1, "plot": 1, "released": 1, "_id": 0}) \
                    .sort("released", 1).limit(5):
    print(f"{m['title']} was released in {m['released']}")
    print(f"Plot: {m['plot']}\n")

Nausicaè of the Valley of the Wind was released in 1984-03-11 00:00:00
Plot: Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.

Nausicaè of the Valley of the Wind was released in 1984-03-11 00:00:00
Plot: Warrior/pacifist Princess Nausicaè desperately struggles to prevent two warring nations from destroying themselves and their dying planet.

Heaven and Earth was released in 1991-02-08 00:00:00
Plot: Warlords Kagetora and Takeda each wish to prevent the other from gaining hegemony in feudal Japan. The two samurai leaders pursue one another across the countryside, engaging in massive ...

Under the Stars was released in 2007-06-15 00:00:00
Plot: Warning! This synopsis contains spoilers Bajo las estrellas (beneath the stars) features the selfish...

Aliens vs. Predator: Requiem was released in 2007-12-25 00:00:00
Plot: Warring alien and predator races descend on a small town, where unsuspecting resid

### Question 2

Question: Group by `rated` and count the number of movies in each.

Answer:

In [9]:
stage_group_rated = {
    "$group": {
        "_id": "$rated",
        "movie_count": {"$sum": 1}
    }
}
stage_sort_rated = {
    "$sort" : {"movie_count" : -1}
}
pipeline = [stage_group_rated,
            stage_sort_rated
            ]

results = movies.aggregate(pipeline)

for result in results:
    print(result)

{'_id': None, 'movie_count': 9894}
{'_id': 'R', 'movie_count': 5537}
{'_id': 'PG-13', 'movie_count': 2321}
{'_id': 'PG', 'movie_count': 1852}
{'_id': 'APPROVED', 'movie_count': 709}
{'_id': 'G', 'movie_count': 477}
{'_id': 'PASSED', 'movie_count': 181}
{'_id': 'TV-14', 'movie_count': 89}
{'_id': 'TV-PG', 'movie_count': 76}
{'_id': 'TV-MA', 'movie_count': 60}
{'_id': 'TV-G', 'movie_count': 59}
{'_id': 'GP', 'movie_count': 44}
{'_id': 'M', 'movie_count': 37}
{'_id': 'Approved', 'movie_count': 5}
{'_id': 'AO', 'movie_count': 3}
{'_id': 'TV-Y7', 'movie_count': 3}
{'_id': 'Not Rated', 'movie_count': 1}
{'_id': 'OPEN', 'movie_count': 1}


### Question 3

Question: Count the number of movies with 3 comments or more.

Answer:

In [ ]:
# Look up related documents in the 'comments' collection:
stage_lookup_comments = {
   "$lookup": {
         "from": "comments", 
         "localField": "_id", 
         "foreignField": "movie_id", 
         "as": "related_comments",
   }
}
# Calculate the number of comments for each movie:
stage_comments_count = {
    "$addFields": {
         "comment_count": {
            "$size": "$related_comments"
         }
    }
}
# Match movie documents with at least 3 comment:
stage_match_comments_count = {
   "$match": {
         "comment_count": {
            "$gte": 3
         }
   } 
}
# Count number of movies with more than 3 comments
stage_count = {
   "$count": "total_movies"
   }

pipeline = [
   stage_lookup_comments,
   stage_comments_count,
   stage_match_comments_count,
   stage_count,
]

results = movies.aggregate(pipeline)
for result in results:
    print(f"Number of movies with more than 3 comments: {result['total_movies']}")

In [ ]:
# Another way through looking at colection: Comment
group_by_movie = {
    "$group": {
        "_id": "$movie_id",              # Take each comment's movie_id and group by it
        "comment_count": {"$sum": 1}     # For each group, count how many comments (add 1 for each)
    }
}
filter_movies = {
    "$match": {
        "comment_count": {"$gte": 3}     # Keep only if comment_count >= 3
    }
}
count_movies = {
    "$count": "movies_with_3plus_comments"  # Count remaining documents and name the result
}
pipeline = [
    group_by_movie,    
    filter_movies,     
    count_movies,       
]

results = comments.aggregate(pipeline)
for result in results:
    print(f"Number of movies with more than 3 comments: {result['movies_with_3plus_comments']}")

Number of movies with more than 3 comments: 400


## Submission

- Submit the URL of the GitHub Repository that contains your work to NTU black board.
- Should you reference the work of your classmate(s) or online resources, give them credit by adding either the name of your classmate or URL.